# 1) Titolo e obiettivi

Lezione 28: Progetto End-to-End Unsupervised - Segmentazione clienti e Anomaly Detection

---

## Mappa della lezione

| Sezione | Contenuto | Tempo stimato |
|---------|-----------|---------------|
| 1 | Titolo, obiettivi, contesto business | 5 min |
| 2 | Teoria: workflow progetto unsupervised | 10 min |
| 3 | Schema mentale: 7 fasi del progetto | 5 min |
| 4 | Implementazione: tutte le 7 fasi con codice | 40 min |
| 5 | Esercizio riassuntivo | 10 min |
| 6 | Conclusione operativa | 10 min |
| 7 | Checklist di fine lezione + glossario | 5 min |
| 8 | Changelog didattico | 2 min |

---

## Obiettivi della lezione

Al termine di questa lezione sarai in grado di:

| # | Obiettivo | Verifica |
|---|-----------|----------|
| 1 | Costruire una **pipeline completa unsupervised** | Sai concatenare tutte le fasi? |
| 2 | Applicare **preprocessing robusto** | Sai usare clipping e RobustScaler? |
| 3 | **Scegliere k** con metriche multiple | Sai confrontare Silhouette/CH/DB? |
| 4 | **Profilare i cluster** per il business | Sai calcolare medie per segmento? |
| 5 | **Separare anomalie** prima del clustering | Sai usare IF/LOF come filtro? |
| 6 | **Validare** con algoritmi alternativi | Sai usare ARI per confronto? |

---

## Contesto business: segmentazione clienti

```
DATI CLIENTI:                         OUTPUT ATTESO:

customer_id | recency | frequency     Segmento A: "Clienti dormienti"
     001    |    120  |      2        Segmento B: "Clienti fedeli"
     002    |     10  |     45        Segmento C: "Nuovi attivi"
     003    |     30  |     15        Anomalie: clienti fuori pattern
     ...    |    ...  |    ...

                   ↓ PIPELINE ↓

┌─────────┐  ┌─────────┐  ┌─────────┐  ┌─────────┐  ┌─────────┐
│  Load   │→ │  Scale  │→ │   PCA   │→ │ Cluster │→ │ Profile │
└─────────┘  └─────────┘  └─────────┘  └─────────┘  └─────────┘
                                              ↓
                                       ┌─────────────┐
                                       │   Anomaly   │
                                       │  Detection  │
                                       └─────────────┘
```

---

## Le 7 fasi del progetto

| Fase | Cosa fa | Strumento |
|------|---------|-----------|
| 1 | **Load & Explore** | pandas, describe, isna |
| 2 | **Preprocessing** | clip, RobustScaler |
| 3 | **PCA** | PCA, cumsum varianza |
| 4 | **Scelta k** | Silhouette, CH, DB |
| 5 | **Clustering** | KMeans su PCA |
| 6 | **Profiling** | groupby().mean() |
| 7 | **Anomaly + Validation** | IF, LOF, Agglomerative, ARI |

---

## Perché questo workflow

| Scelta | Motivazione |
|--------|-------------|
| **RobustScaler** | Resistente a outlier (usa mediana/IQR) |
| **PCA prima di cluster** | Distanze più stabili, clustering più veloce |
| **Metriche multiple per k** | Nessuna metrica è perfetta da sola |
| **Anomaly dopo segmentazione** | Evita che outlier distorcano i centroidi |
| **Validazione con altro algoritmo** | Conferma che la struttura è reale |

---

## Dataset RFM simulato

| Feature | Significato | Range tipico |
|---------|-------------|--------------|
| recency | Giorni dall'ultimo acquisto | 0-365 |
| frequency | Numero acquisti | 1-100 |
| monetary | Spesa totale | 10-10000 |
| tenure | Mesi da iscrizione | 1-120 |
| online_ratio | % acquisti online | 0-1 |

**Output atteso:** 3-5 segmenti + 3-5% anomalie isolate.

---

## Prerequisiti

| Concetto | Dove lo trovi | Verifica |
|----------|---------------|----------|
| PCA | Lezione 24 | Sai scegliere n_components? |
| K-Means | Lezione 20 | Sai usare fit_predict? |
| Scelta k | Lezione 21 | Sai interpretare Silhouette? |
| IF, LOF | Lezione 26 | Sai settare contamination? |
| Feature Engineering | Lezione 27 | Sai creare feature da cluster? |

**Tecniche usate:** RobustScaler, PCA, KMeans, AgglomerativeClustering, DBSCAN, IsolationForest, LOF, silhouette_score, calinski_harabasz_score, davies_bouldin_score, adjusted_rand_score.

# 2) Teoria concettuale
- Progetto unsupervised = scoprire struttura nei dati: segmenti e outlier.
- Riduzione dimensionale (PCA) rende piu' stabili distanze e velocizza clustering.
- Scelta del numero di cluster: confrontare silhouette, Calinski-Harabasz, Davies-Bouldin e il contesto business.
- Anomaly detection: separare clienti fuori pattern per non contaminare i segmenti.


# 3) Schema mentale / mappa decisionale
1. Genera o carica i dati e verifica la qualita'.
2. Preprocessing: clipping outlier leggero e scaling robusto.
3. PCA per compattare l'informazione (80-90% varianza) e rendere stabili le distanze.
4. Scelta del numero di cluster con piu' metriche su PCA.
5. Clustering principale e calcolo profili.
6. Rilevazione anomalie per separare punti sospetti.
7. Validazione con algoritmi alternativi e preparazione deliverable.


# 4) Sezione dimostrativa
Fasi del progetto implementate di seguito (tutte commentate e con checkpoint):
- Fase 2: data loading ed esplorazione.
- Fase 3: preprocessing e scaling.
- Fase 4: PCA e varianza spiegata.
- Fase 5: scelta del numero di cluster.
- Fase 6: clustering con KMeans e profili.
- Fase 7: anomaly detection.
- Fase 8: validazione con altri algoritmi.


In [ ]:
# Fase 2: Data loading & exploration
# Scopo: generare un dataset clienti simulato, controllare NaN e statistiche base.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)
plt.close('all')

n = 1200
recency = np.random.exponential(scale=35, size=n)
frequency = np.random.poisson(lam=6, size=n)
monetary = np.random.gamma(shape=2.5, scale=80, size=n)
avg_basket = monetary / (frequency + 1)
tenure = np.random.uniform(3, 60, size=n)
online_ratio = np.random.beta(a=2, b=5, size=n)

df = pd.DataFrame({
    'recency': recency,
    'frequency': frequency,
    'monetary': monetary,
    'avg_basket': avg_basket,
    'tenure': tenure,
    'online_ratio': online_ratio
})
print(df.head())
print(df.describe())
assert not df.isna().any().any(), "Ci sono NaN nel dataset simulato"


In [ ]:
# Fase 3: preprocessing
# Scopo: ridurre effetto outlier con clipping e scalare con RobustScaler per robustezza.
from sklearn.preprocessing import RobustScaler, StandardScaler

# Clipping leggero al 1-99 percentile per gestire code lunghe
clipped = df.copy()
for col in clipped.columns:
    lower, upper = np.percentile(clipped[col], [1, 99])
    clipped[col] = clipped[col].clip(lower, upper)

scaler = RobustScaler()
X_scaled = scaler.fit_transform(clipped)
print(f"Shape dopo scaling: {X_scaled.shape}")
assert X_scaled.shape[0] == df.shape[0], "Shape incoerente dopo scaling"


In [ ]:
# Fase 4: PCA
# Scopo: ridurre dimensionalita' mantenendo ~90% varianza e preparare dati per clustering.
from sklearn.decomposition import PCA

pca = PCA(n_components=0.90, random_state=42)
X_pca = pca.fit_transform(X_scaled)
var_cum = pca.explained_variance_ratio_.sum()
print(f"Componenti PCA: {X_pca.shape[1]}, Varianza cumulata: {var_cum:.3f}")
assert var_cum >= 0.85, "Varianza spiegata troppo bassa"

fig, ax = plt.subplots(figsize=(6,4))
ax.plot(np.cumsum(pca.explained_variance_ratio_), marker='o')
ax.set_ylabel('Varianza cumulata')
ax.set_xlabel('Numero componenti')
ax.axhline(0.90, color='gray', linestyle='--')
ax.set_title('Scree plot cumulativo PCA')
plt.tight_layout()
plt.show()


In [ ]:
# Fase 5: scelta del numero di cluster
# Scopo: confrontare k=2..6 usando silhouette, Calinski-Harabasz e Davies-Bouldin su PCA.
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

results = []
for k in range(2, 7):
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_pca)
    sil = silhouette_score(X_pca, labels)
    ch = calinski_harabasz_score(X_pca, labels)
    db = davies_bouldin_score(X_pca, labels)
    results.append({'k': k, 'silhouette': sil, 'calinski_harabasz': ch, 'davies_bouldin': db})

res_df = pd.DataFrame(results)
print(res_df)
best_k = res_df.sort_values(by='silhouette', ascending=False).iloc[0]['k']
print(f"k scelto (silhouette): {int(best_k)}")
assert best_k >= 2, "k non valido"


In [ ]:
# Fase 6: clustering con KMeans
# Scopo: applicare KMeans con k scelto e creare profili di cluster.
best_k = int(best_k)
km_final = KMeans(n_clusters=best_k, random_state=42, n_init=10)
labels_final = km_final.fit_predict(X_pca)

sil_final = silhouette_score(X_pca, labels_final)
ch_final = calinski_harabasz_score(X_pca, labels_final)
db_final = davies_bouldin_score(X_pca, labels_final)
print(f"Silhouette: {sil_final:.3f}, CH: {ch_final:.1f}, DB: {db_final:.3f}")

# Dataset arricchito
proj_df = df.copy()
proj_df['cluster'] = labels_final
proj_df['pc1'] = X_pca[:,0]
proj_df['pc2'] = X_pca[:,1] if X_pca.shape[1] > 1 else 0
cluster_profiles = proj_df.groupby('cluster').mean()
print(cluster_profiles)
assert proj_df['cluster'].nunique() == best_k, "Numero cluster incoerente"


In [ ]:
# Fase 7: anomaly detection
# Scopo: identificare clienti atipici per non contaminarli con cluster sbagliati.
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

iso = IsolationForest(contamination=0.03, random_state=42)
iso_preds = iso.fit_predict(X_scaled)
iso_flag = (iso_preds == -1).astype(int)

lof = LocalOutlierFactor(n_neighbors=25, contamination=0.03)
lof_preds = lof.fit_predict(X_scaled)
lof_flag = (lof_preds == -1).astype(int)

anom_flag = ((iso_flag + lof_flag) > 0).astype(int)
proj_df['anomaly_flag'] = anom_flag
print(proj_df['anomaly_flag'].value_counts())
assert proj_df['anomaly_flag'].sum() > 0, "Nessuna anomalia rilevata"


In [ ]:
# Fase 8: validazione con algoritmi alternativi
# Scopo: confrontare Agglomerative e DBSCAN come stress test dei cluster.
from sklearn.cluster import AgglomerativeClustering, DBSCAN
from sklearn.metrics import adjusted_rand_score

agg = AgglomerativeClustering(n_clusters=best_k)
labels_agg = agg.fit_predict(X_pca)

# DBSCAN su PCA 2D
dbscan = DBSCAN(eps=1.2, min_samples=8)
labels_db = dbscan.fit_predict(X_pca[:, :2])

rows = []
rows.append({'modello': 'Agglomerative', 'silhouette': silhouette_score(X_pca, labels_agg), 'ari_vs_kmeans': adjusted_rand_score(labels_final, labels_agg)})
mask = labels_db != -1
if mask.sum() > 0 and np.unique(labels_db[mask]).size > 1:
    sil_db = silhouette_score(X_pca[mask], labels_db[mask])
else:
    sil_db = np.nan
rows.append({'modello': 'DBSCAN', 'silhouette': sil_db, 'ari_vs_kmeans': adjusted_rand_score(labels_final, labels_db)})

val_df = pd.DataFrame(rows)
print(val_df)

# Deliverable: dataset arricchito pronto per analisi/report
print(proj_df.head())
assert proj_df.shape[0] == n, "Shape finale incoerente"


# 5) Esercizi svolti (passo-passo)
Esercizio unico: replica la pipeline e verifica ad ogni fase le attese (NaN, varianza, metriche, profili, anomalie). Usa le check-list indicate nei commenti del codice.


# 6) Conclusione operativa

## 5 take-home messages

| # | Messaggio | Perché importante |
|---|-----------|-------------------|
| 1 | **Preprocessing robusto** | RobustScaler + clipping protegge da outlier |
| 2 | **PCA stabilizza clustering** | Distanze più significative in bassa D |
| 3 | **Metriche multiple per k** | Silhouette + CH + DB + business sense |
| 4 | **Profili interpretativi** | Dai nomi ai cluster per stakeholder |
| 5 | **Validazione incrociata** | ARI con altro algoritmo conferma struttura |

---

## Workflow riassuntivo del progetto

```
┌──────────────────────────────────────────────────────────────────────────┐
│                    PIPELINE UNSUPERVISED END-TO-END                      │
├──────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   FASE 1: Load          df = pd.read_csv(...) + info() + describe()     │
│      ↓                                                                   │
│   FASE 2: Preprocess    clip outlier + RobustScaler                      │
│      ↓                                                                   │
│   FASE 3: PCA           n_components → 90% varianza                      │
│      ↓                                                                   │
│   FASE 4: Scelta k      Loop k=2..10, confronta Sil/CH/DB               │
│      ↓                                                                   │
│   FASE 5: Cluster       KMeans(k_best).fit_predict(X_pca)               │
│      ↓                                                                   │
│   FASE 6: Profile       df.groupby('cluster').mean()                    │
│      ↓                                                                   │
│   FASE 7: Anomaly       IF/LOF → flag anomalie                          │
│      ↓                                                                   │
│   FASE 8: Validate      Agglomerative/DBSCAN → ARI vs KMeans            │
│                                                                          │
└──────────────────────────────────────────────────────────────────────────┘
```

---

## Metodi spiegati: reference card progetto

| Metodo | Fase | Input | Output |
|--------|------|-------|--------|
| `RobustScaler` | 2 | X raw | X scalato (mediana, IQR) |
| `PCA(n_components)` | 3 | X scalato | X_pca, varianza spiegata |
| `silhouette_score` | 4 | X, labels | score (-1..1) |
| `calinski_harabasz_score` | 4 | X, labels | score (alto = buono) |
| `davies_bouldin_score` | 4 | X, labels | score (basso = buono) |
| `KMeans(k)` | 5 | X_pca | labels, centroids |
| `groupby().mean()` | 6 | df con label | profili segmenti |
| `IsolationForest` | 7 | X | anomaly labels |
| `adjusted_rand_score` | 8 | labels1, labels2 | ARI (-1..1) |

---

## Errori comuni e debug rapido

| Errore | Perché sbagliato | Fix |
|--------|-----------------|-----|
| Silhouette bassa (<0.2) | k sbagliato o dati non separabili | Prova k diversi, verifica PCA |
| Nessuna anomalia | contamination troppo bassa | Aumenta a 0.03-0.05 |
| DBSCAN trova 1 solo cluster | eps troppo grande | Usa k-distance plot per eps |
| Profili tutti simili | Cluster non discriminanti | Rivedi k o feature |
| ARI vicino a 0 | Algoritmi non concordano | Struttura debole, aumenta dati |

---

## Template deliverable progetto

```python
# Output finale del progetto:
# 1. df_segmented: DataFrame con colonna 'segment' e profili
# 2. df_anomalies: DataFrame con anomalie isolate
# 3. report_metrics: dict con Silhouette, CH, DB, ARI
# 4. fig_profiles: grafico radar o barplot dei profili

# Esempio export:
df_segmented.to_csv('clienti_segmentati.csv', index=False)
df_anomalies.to_csv('clienti_anomali.csv', index=False)

import json
with open('metriche_progetto.json', 'w') as f:
    json.dump(report_metrics, f, indent=2)
```

---

## Prossimi passi

| Lezione | Argomento | Collegamento |
|---------|-----------|--------------|
| 29 | Fondamenti AI | Passaggio a ML/AI concetti |
| 30+ | NLP, Deep Learning | Estensioni avanzate |

**Fine modulo Unsupervised!** Hai completato clustering, PCA, anomaly detection e progetto end-to-end.

# 7) Checklist di fine lezione
- [ ] Ho controllato NaN e outlier prima del clustering.
- [ ] Ho scalato e ridotto con PCA verificando la varianza cumulata.
- [ ] Ho scelto k confrontando piu' metriche e ho documentato la scelta.
- [ ] Ho calcolato profili di cluster interpretabili (recency/frequency/monetary/... ).
- [ ] Ho eseguito anomaly detection separando i punti anomali.
- [ ] Ho validato con almeno un algoritmo alternativo (Agglomerative/DBSCAN).

Glossario
- RFM: Recency, Frequency, Monetary, base della segmentazione clienti.
- PCA: riduzione dimensionale preservando varianza.
- Silhouette: misura coesione/separazione cluster (-1..1).
- Calinski-Harabasz / Davies-Bouldin: metriche di qualita' cluster (alto CH, basso DB).
- KMeans: clustering a centroidi con k fissato.
- Agglomerative: clustering gerarchico bottom-up.
- DBSCAN: clustering per densita' con etichetta -1 per rumore.
- IsolationForest / LOF: algoritmi di anomaly detection.
- Contamination: percentuale attesa di anomalie.
- Profilo di cluster: medie/quantili delle feature per ogni cluster.


# 8) Changelog didattico

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | 2024-01-25 | Creazione: pipeline base segmentazione |
| 1.1 | 2024-02-05 | Aggiunta anomaly detection e profiling |
| 2.0 | 2024-02-12 | Integrata validazione con ARI |
| 2.1 | 2024-02-18 | Refactor con controlli e checkpoint |
| **2.3** | **2024-12-19** | **ESPANSIONE COMPLETA:** mappa lezione 8 sezioni, tabella obiettivi, ASCII pipeline 7 fasi, contesto business RFM, 5 take-home messages, workflow diagramma completo, reference card metodi per fase, template deliverable export, errori comuni con fix |

---

## Note per lo studente

Questa lezione conclude il **modulo Unsupervised Learning**:

| Lesson | Argomento | Stato |
|--------|-----------|-------|
| 19 | Intro Unsupervised | COMPLETATO |
| 20 | K-Means | COMPLETATO |
| 21 | Scelta K | COMPLETATO |
| 22 | Gerarchico | COMPLETATO |
| 23 | DBSCAN | COMPLETATO |
| 24 | PCA | COMPLETATO |
| 25 | PCA + Clustering | COMPLETATO |
| 26 | Anomaly Detection | COMPLETATO |
| 27 | Feature Engineering | COMPLETATO |
| **28** | **Progetto End-to-End** | **COMPLETATO** |

**Hai ora le competenze per:**
- Segmentare dataset senza label
- Scegliere algoritmi e iperparametri con criterio
- Validare e consegnare risultati al business

**Prossimo modulo:** AI, NLP, Deep Learning (Lessons 29+)